In [2]:
import os
import zipfile
import random, shutil
from google.colab import drive

drive.mount('/content/drive/')
zip_ref = zipfile.ZipFile("/content/drive/My Drive/cats_vs_dogs_subset.zip", 'r')
zip_ref.extractall("/tmp")
zip_ref.close()

Mounted at /content/drive/


In [3]:
import pathlib
import keras
from keras import layers

original_dir = pathlib.Path("/tmp/cats-dogs-images")
new_base_dir = pathlib.Path("/tmp/cats_vs_dogs_subset")

from keras.utils import image_dataset_from_directory

train_dataset = image_dataset_from_directory(
    new_base_dir / "train",
    image_size=(180, 180),
    batch_size=32
)
validation_dataset = image_dataset_from_directory(
    new_base_dir / "validation",
    image_size=(180, 180),
    batch_size=32
)
test_dataset = image_dataset_from_directory(
    new_base_dir / "test",
    image_size=(180, 180),
    batch_size=32
)

data_augmentation = keras.Sequential([
    layers.RandomFlip("horizontal"), # Applies horizontal flipping to a random 50% of the images that go through it
    layers.RandomRotation(0.2), # Rotates the input images by a random value in the range [–10%, +10%] | [–36 degrees, +36 degrees]
    layers.RandomZoom(0.2) # Zooms in or out of the image by a random factor in the range [-20%, +20%]
])

Found 2000 files belonging to 2 classes.
Found 1000 files belonging to 2 classes.
Found 2000 files belonging to 2 classes.


In [4]:
inputs = keras.Input(shape=(180, 180, 3))
x = data_augmentation(inputs)
x = layers.Rescaling(1./255)(x)
# the first layer in our model is a regular Conv2D layer. We’ll start using SeparableConv2D afterwards.
# when we do BatchNormalization and put activation after BatchNormalization we do use_bias=False
x = layers.Conv2D(filters=32, kernel_size=5, use_bias=False)(x)

# We apply a series of convolutional blocks with increasing feature depth.
# Each block consists of two batch-normalized depthwise separable convolution layers
# and a max pooling layer, with a residual connection around the entire block.
for size in [32, 64, 128, 256, 512]:
    residual = x

    x = layers.BatchNormalization()(x)
    x = layers.Activation("relu")(x)
    x = layers.SeparableConv2D(size, 3, padding="same", use_bias=False)(x)

    x = layers.BatchNormalization()(x)
    x = layers.Activation("relu")(x)
    x = layers.SeparableConv2D(size, 3, padding="same", use_bias=False)(x)

    x = layers.MaxPooling2D(size, strides=2, padding="same")(x)

    # If we use MaxPooling, we use a strided convolution to project the residual to correct shape
    residual = layers.Conv2D(size, 1, strides=2, padding="same", use_bias=False)(residual)
    x = layers.add([x, residual])

# In the original model, we used a Flatten layer before the Dense layer. Here, we go with a GlobalAveragePooling2D layer.
x = layers.GlobalAveragePooling2D()(x)
x = layers.Dropout(0.5)(x) # Like in the original model, we add a dropout layer for regularization.
outputs = layers.Dense(1, activation="sigmoid")(x)
model = keras.Model(inputs=inputs, outputs=outputs)

In [ ]:
model.compile(
    optimizer="rmsprop",
    loss="binary_crossentropy",
    metrics=["accuracy"]
)

callbacks = [
    keras.callbacks.ModelCheckpoint(
    filepath="mini-xception-model.keras",
    save_best_only=True,
    monitor="val_loss")
]

history = model.fit(
    train_dataset,
    epochs=50,
    validation_data=validation_dataset,
    callbacks=callbacks
)

Epoch 1/50
63/63 ━━━━━━━━━━━━━━━━━━━━ 98s 1s/step - accuracy: 0.5304 - loss: 0.8805 - val_accuracy: 0.5010 - val_loss: 0.6929
Epoch 2/50
63/63 ━━━━━━━━━━━━━━━━━━━━ 138s 1s/step - accuracy: 0.6062 - loss: 0.6588 - val_accuracy: 0.4980 - val_loss: 0.6924
Epoch 3/50
63/63 ━━━━━━━━━━━━━━━━━━━━ 142s 1s/step - accuracy: 0.6387 - loss: 0.6432 - val_accuracy: 0.5000 - val_loss: 0.6938
Epoch 4/50
63/63 ━━━━━━━━━━━━━━━━━━━━ 87s 1s/step - accuracy: 0.6785 - loss: 0.6128 - val_accuracy: 0.5400 - val_loss: 0.6904
Epoch 5/50
63/63 ━━━━━━━━━━━━━━━━━━━━ 142s 1s/step - accuracy: 0.6897 - loss: 0.5803 - val_accuracy: 0.5000 - val_loss: 0.6954
Epoch 6/50
63/63 ━━━━━━━━━━━━━━━━━━━━ 87s 1s/step - accuracy: 0.7104 - loss: 0.5527 - val_accuracy: 0.6440 - val_loss: 0.6588
Epoch 7/50
63/63 ━━━━━━━━━━━━━━━━━━━━ 142s 1s/step - accuracy: 0.7214 - loss: 0.5463 - val_accuracy: 0.5790 - val_loss: 0.6479
Epoch 8/50
63/63 ━━━━━━━━━━━━━━━━━━━━ 142s 1s/step - accuracy: 0.7224 - loss: 0.5371 - val_accuracy: 0.5520 - val_